In [2]:
import numpy as np
import pandas as pd
import json
import pickle
import ast

In [3]:
movies=pd.read_csv('DataSets/movies_metadata.csv')

/home/abuzar/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [5]:
columns=['id','original_title','genres','vote_average', 'vote_count']

In [6]:
Final_data=movies[columns]

In [7]:
Final_data.dropna()

,id,original_title,genres,vote_average,vote_count
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",7.7,5415.0
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",6.9,2413.0
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",6.5,92.0
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",6.1,34.0
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",5.7,173.0
...,...,...,...,...,...
45457,439050,رگ خواب,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",4.0,1.0
45458,111109,Siglo ng Pagluluwal,"[{'id': 18, 'name': 'Drama'}]",9.0,3.0
45459,67758,Betrayal,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",3.8,6.0
45460,227506,Satana likuyushchiy,[],0.0,0.0


In [8]:
def ConvertToDict(ColumnsList,Resultant,Size):
    for i in range (0,Size,1):
        new_list=[]
        dict1=[]
        found=False
        for j in range(0,len(ColumnsList[i]),1):
            if (ColumnsList[i][j]=='{'):
                found=True
            elif ColumnsList[i][j]=='}':
                if ColumnsList[i][j]:
                    new_list+=ColumnsList[i][j]
                found=False
                list1=''.join(new_list)
                dict1.append(ast.literal_eval(list1))
                new_list=[]

            if found==True:
                if ColumnsList[i][j]:
                    new_list+=ColumnsList[i][j]
        Resultant.append(dict1)
    return Resultant

In [9]:
movie_list=Final_data['genres']
FinalMovieList=[]
FinalMovieList=ConvertToDict(movie_list,FinalMovieList,len(movie_list))


In [10]:
def ReturnList(RawList):
    FGenres=[]
    for i in range(0,len(RawList)):
        temp=[]
        for j in range(0,len(RawList[i])):
            temp.append(RawList[i][j]['name'])
        FGenres.append(temp)
    return FGenres

In [11]:
GenList=ReturnList(FinalMovieList)
Final_data.insert(3, "GenList",GenList, True)


In [15]:
with open("Genres.pkl", "rb") as f:
    Gen1 = pickle.load(f)
    f.close()

In [16]:
Final_data=Final_data.drop(Final_data.columns[2],axis=1)

Applying non-personal approch for the new users.
Using formula for weighted averages ued by imdb.

In [17]:
VoteAvgMean=Final_data['vote_average'].mean()

In [18]:
MinVoteReq=Final_data['vote_count'].quantile(0.95)

In [19]:
Popular_Movies=Final_data.copy().loc[Final_data['vote_count']>=MinVoteReq]

In [20]:
Popular_Movies

,id,original_title,GenList,vote_average,vote_count
0,862,Toy Story,"[Animation, Comedy, Family]",7.7,5415.0
1,8844,Jumanji,"[Adventure, Fantasy, Family]",6.9,2413.0
5,949,Heat,"[Action, Crime, Drama, Thriller]",7.7,1886.0
9,710,GoldenEye,"[Adventure, Action, Thriller]",6.6,1194.0
15,524,Casino,"[Drama, Crime]",7.8,1343.0
...,...,...,...,...,...
44620,406990,What Happened to Monday,"[Science Fiction, Thriller]",7.3,598.0
44628,341013,Atomic Blonde,"[Action, Thriller]",6.1,748.0
44674,374720,Dunkirk,"[Action, Drama, History, Thriller, War]",7.5,2712.0
44838,335988,Transformers: The Last Knight,"[Action, Science Fiction, Thriller, Adventure]",6.2,1440.0


Applying Formlation on Popular_Movies dataframe

In [21]:
def Scores(DataFrame,VoteAvgMean,MinVoteReq):
    Num_votes=DataFrame['vote_count']
    Avg_rating=DataFrame['vote_average']
    scores=[]
    for i in range (0,len(Num_votes),1):
        scoreTemp=0.0
        scoreTemp=((((Num_votes.iloc[i])/(Num_votes.iloc[i]+MinVoteReq))*Avg_rating.iloc[i])
                   +(((MinVoteReq/(Num_votes.iloc[i]+MinVoteReq))*VoteAvgMean)))
        scores.append(scoreTemp)
    return scores

In [22]:
Popular_Movies.insert(5, "Scores",Scores(Popular_Movies,VoteAvgMean,MinVoteReq), True)

In [23]:
Popular_Movies=Popular_Movies.sort_values('Scores',ascending=False)

In [57]:
def NormalPridiction(Final_data):
    VoteAvgMean=Final_data['vote_average'].mean()
    MinVoteReq=Final_data['vote_count'].quantile(0.95)
    Popular_Movies=Final_data.copy().loc[Final_data['vote_count']>=MinVoteReq]
    Popular_Movies.insert(5, "Scores",Scores(Popular_Movies,VoteAvgMean,MinVoteReq), True)
    Popular_Movies.insert(4, "For Genre","Null", True)
    Popular_Movies=Popular_Movies.sort_values('Scores',ascending=False)
    return Popular_Movies

In [46]:
def FilteringGenres(Final_data,GenreName):   
    PopularMoviesWithGenres=[]
    for i in range (0,len(Final_data)):
        for j in range (0,len(Final_data['GenList'].iloc[i])):
            if (Final_data['GenList'].iloc[i][j]==GenreName):
                PopularMoviesWithGenres.append(Final_data.iloc[i])
    return PopularMoviesWithGenres

In [53]:
def GenreFiltrationPridiction(Final_data,GenreName):
    Filtered_Genres=FilteringGenres(Final_data,GenreName)
    PopularMoviesGenres=pd.DataFrame(Filtered_Genres)
    VoteAvgMean=Final_data['vote_average'].mean()
    MinVoteReqGen=Final_data['vote_count'].quantile(0.95)
    PopularMoviesGenresV2=PopularMoviesGenres.copy().loc[PopularMoviesGenres['vote_count']>=MinVoteReqGen]
    PopularMoviesGenresV2.insert(5, "Scores",Scores(PopularMoviesGenresV2,VoteAvgMean,MinVoteReqGen), True)
    PopularMoviesGenresV2.insert(4, "For Genre",GenreName, True)
    PopularMoviesGenresV2=PopularMoviesGenresV2.sort_values('Scores',ascending=False)
    return PopularMoviesGenresV2

In [65]:
Pop_Movies=NormalPridiction(Final_data).head(100)
for i in range (0,len(Gen1)):
    df=GenreFiltrationPridiction(Final_data,Gen1[i])
    Pop_Movies=pd.concat([Pop_Movies,df.head(100)], axis=0)


In [63]:
Pop_Movies

,id,original_title,GenList,vote_average,For Genre,vote_count,Scores
314,278,The Shawshank Redemption,"[Drama, Crime]",8.5,Null,8358.0,8.357744
834,238,The Godfather,"[Drama, Crime]",8.5,Null,6024.0,8.306331
12480,155,The Dark Knight,"[Drama, Action, Crime, Thriller]",8.3,Null,12269.0,8.208375
2843,550,Fight Club,[Drama],8.3,Null,9678.0,8.184898
292,680,Pulp Fiction,"[Thriller, Crime]",8.3,Null,8670.0,8.172154
...,...,...,...,...,...,...,...
45010,353491,The Dark Tower,"[Action, Western, Science Fiction, Fantasy, Ho...",5.7,Western,688.0,5.668347
16491,49849,Cowboys & Aliens,"[Action, Science Fiction, Thriller, Western]",5.4,Western,2054.0,5.438057
2586,8487,Wild Wild West,"[Action, Adventure, Comedy, Science Fiction, W...",5.1,Western,1042.0,5.252362
11033,10947,High School Musical,"[Comedy, Drama, Family, Music, TV Movie]",6.1,TV Movie,1048.0,5.958897


In [64]:
Pop_Movies.to_csv('NonPersonalFilteringResults.csv',index=False)